# OISSTv2 Live Updating Workflow

This notebook is for documenting the steps to build a single annual OISSTv2 file using update caches. The processing of global anomalies, and the regional timelines that we keep on hand for analyses. 

The steps were developed initially in `Update_01_Download_OISST` then the steps were abstracted into functions and placed into the `oisstools.py` module. This notebook will follow the same steps but with much less code, hopefully with less room for error.

This notebook exists to document the processing steps, but has since been replaced by script workflows using prefect in `oisst_prefect.py`. Use this notebook for debugging or extending the workflow, then transfer to `oisst_prefect.py`

In [1]:
# Libraries
from bs4 import BeautifulSoup
import requests
import os
import dask
import xarray as xr
import datetime
import oisstools as ot


#### 1. Set workspace
workspace = "local"
box_root = ot.set_workspace("local")
_cache_root = ot.set_cache_root(box_root)

# Manual Month/Year Selection
# last_month = "11"
# this_month = "12"
# update_yr = "2022"


#### 2. What Months are we checking?
this_month = ot.get_update_month(return_this_month = True)
last_month = ot.get_update_month(return_this_month = False)
update_yr  = ot.check_update_yr(for_this_month = True)


# Specify year range to update for anomaly timeseries
yr_min = update_yr
yr_max = update_yr


# variable name to mask and make timeline from
var_name = "sst"


#### 3. Print Last Update
print(f"Working via directory at: {box_root}")
print(f"Updated using months:     {last_month} & {this_month}, Year: {update_yr}")
print(f"Updating Year(s):         {yr_min} - {yr_max}")

Working via directory at: /Users/akemberling/Library/CloudStorage/Box-Box/
Updated using months:     02 & 03, Year: 2022
Updating Year(s):         2022 - 2022


## 1. Download Daily Files for Last Month

In [2]:
# Check that the root directory isn't broken
os. listdir('/Users/akemberling/Library/CloudStorage/') 

['.DS_Store', 'Box-Box']

In [3]:
# Use cache_oisst function to update cache for last month
if last_month != "12":
    ot.cache_oisst(cache_month = last_month, 
                   update_yr = update_yr, 
                   workspace = workspace, 
                   verbose = True)
else:
    print("Last month not updated because of year transition")    

Caching Daily NETCDF File: oisst-avhrr-v02r01.20220201.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220202.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220203.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220204.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220205.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220206.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220207.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220208.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220209.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220210.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220211.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220212.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220213.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220214.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220215.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220216.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220217.nc
Caching Daily 

## 2. Download Daily Files for This Month

In [4]:
# Use cache_oisst function to update cache for current month
ot.cache_oisst(cache_month = this_month, 
               update_yr = update_yr, 
               workspace = workspace, 
               verbose = True)

Caching Daily NETCDF File: oisst-avhrr-v02r01.20220301.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220302.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220303.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220304.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220305.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220306.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220307.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220308.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220309_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220310_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220311_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220312_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220313_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220314_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.20220315_preliminary.nc
Caching Daily NETCDF File: oisst-avhrr-v02r01.

## 3. Build Annual File

In [5]:
# Assemble the annual file
oisst_update = ot.build_annual_from_cache(last_month = last_month, 
                                          this_month = this_month, 
                                          workspace = workspace, 
                                          verbose = True)

## 4. Check Annual file

Want to see consistency in the dimensions, and data from the beginning of the year through to the update month.

In [6]:
#oisst_update
oisst_update.sst.mean(("lon", "lat")).plot()

## 5. Export Update

In [7]:
# export update using same file pattern as everything else
ot.export_annual_update(cache_root   = _cache_root,
                        update_yr    = update_yr,
                        oisst_update = oisst_update)

File Saved to /Users/akemberling/Library/CloudStorage/Box-Box/RES_Data/OISST/oisst_mainstays/annual_observations/sst.day.mean.2022.v2.nc


## 6. Update Annual Anomalies

This step has the opportunity to update anomalies for a range of years, the default update year is the `update_yr` value set at the top of the notbeook.

In [8]:
# Set desired climatology period
reference_period = "1982-2011"
print(f"Calculating Anomalies for {reference_period} reference period.")

# Process Global Anomalies for the select years
ot.update_global_anomalies(yr_min = 2020, 
                           yr_max = yr_max, 
                           box_root = box_root, 
                           var_name = "sst", 
                           reference_period = "1982-2011")

Calculating Anomalies for 1982-2011 reference period.
Saving anomaly year: 2020
Saving anomaly year: 2021
Saving anomaly year: 2022


## 7. Update Global Timeseries

In [9]:
# Update the timeseries of global temps/anoms
ot.update_global_timeseries(yr_min, yr_max, box_root, var_name = "sst", reference_period = "1982-2011")

Updating Global Timeseries


## 8. Regional timeseries 

In [10]:
#### Update the regional timeseries groups: ####

# GMRI Focal Areas
ot.update_regional_timeseries_collection(start_yr = yr_min, end_yr = yr_max, region_collection = "gmri_sst_focal_areas", box_root = box_root)

# Northeast Marine Regions
ot.update_regional_timeseries_collection(start_yr = yr_min, end_yr = yr_max, region_collection = "nelme_regions", box_root = box_root)

# NMFS Trawl Regions
ot.update_regional_timeseries_collection(start_yr = yr_min, end_yr = yr_max, region_collection = "nmfs_trawl_regions", box_root = box_root)

# Gulf of Maine Physio Regions
ot.update_regional_timeseries_collection(start_yr = yr_min, end_yr = yr_max, region_collection = "gom_physio_regions", box_root = box_root)

# Large Marine Ecosystems
ot.update_regional_timeseries_collection(start_yr = yr_min, end_yr = yr_max, region_collection = "lme", box_root = box_root)

Updating Timeseries for: gmri_sst_focal_areas
Updating Timeseries for:
 - apershing_gulf_of_maine
 - cpr_gulf_of_maine
 - aak_northwest_atlantic
 - long_island_sound
Updating Timeseries for: nelme_regions
Updating Timeseries for:
 - GoM
 - NELME
 - SNEandMAB


/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/regionmask/core/regions.py:410: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in poly:
/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/regionmask/core/regions.py:410: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in poly:
/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/regionmask/core/regions.py:410: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in poly:


Updating Timeseries for: nmfs_trawl_regions
Updating Timeseries for:
 - georges_bank
 - gulf_of_maine
 - southern_new_england
 - mid_atlantic_bight
 - inuse_strata
 - regions_collection
Updating Timeseries for: gom_physio_regions
Updating Timeseries for:
 - bay_of_fundy
 - bear_seamount
 - browns_bank
 - central_gulf_of_maine
 - continental_slope
 - eastern_coastal_shelf
 - georges_bank
 - georges_basin
 - jordan_basin
 - kelvin_seamount
 - manning_seamount
 - northern_coastal_shelf
 - scotian_coastal_shelf
 - scotian_shelf
 - southern_coastal_shelf
 - wikinson_basin


/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/regionmask/core/regions.py:410: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in poly:
/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/regionmask/core/regions.py:410: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in poly:
/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/regionmask/core/regions.py:410: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in poly:
/Users/akemberling/opt/anaconda3/envs

Updating Timeseries for: lme
Updating Timeseries for:
 - agulhas_current
 - aleutian_islands
 - antarctica
 - arabian_sea
 - baltic_sea
 - barents_sea
 - bay_of_bengal
 - beaufort_sea
 - benguela_current
 - black_sea
 - california_current
 - canadian_eastern_arctic_west_greenland
 - canadian_high_arctic_north_greenland
 - canary_current
 - caribbean_sea
 - celtic_biscay_shelf
 - central_arctic
 - east_bering_sea
 - east_brazil_shelf
 - east_central_australian_shelf
 - east_china_sea
 - east_siberian_sea
 - faroe_plateau
 - greenland_sea
 - guinea_current
 - gulf_of_alaska
 - gulf_of_california
 - gulf_of_mexico
 - gulf_of_thailand
 - hudson_bay_complex
 - humboldt_current
 - iberian_coastal
 - iceland_shelf_and_sea
 - indonesian_sea
 - insular_pacific_hawaiian
 - kara_sea
 - kuroshio_current
 - labrador_newfoundland
 - laptev_sea
 - mediterranean_sea
 - new_zealand_shelf
 - north_australian_shelf
 - north_brazil_shelf
 - north_sea
 - northeast_australian_shelf
 - northeast_us_continent

---

# 9. TESTING: Make Monthly Files

Debugging: Having errors with how time is stored in 2021 vs. other years

In [4]:
# Debugging a single year:
temp_20 = ot.load_box_oisst(box_root, start_yr = 2021, end_yr = 2021, anomalies = False, do_parallel = True)
temp_20.coords
temp_21 = ot.load_box_oisst(box_root, start_yr = 2021, end_yr = 2021, anomalies = False, do_parallel = True)
temp_21.coords

xr.concat([temp_20, temp_21], dim = "time").coords

Coordinates:
  * time     (time) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-12-31
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9

Running all the years through:

In [6]:
# Load all the daily files we have
oisst_temp_daily = ot.load_box_oisst(box_root, start_yr = 1981, end_yr = update_yr, anomalies = False, do_parallel = True)
oisst_anoms_daily = ot.load_box_oisst(box_root, start_yr = 1981, end_yr = update_yr, anomalies = True, do_parallel = True)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/akemberling/Library/CloudStorage/Box-Box/RES_Data/OISST/oisst_mainstays/annual_anomalies/1982to2011_climatology/daily_anoms_2020.nc'

In [ ]:
# Use resample to get monthly averages for each year - keeps index of time
oisst_months = oisst_temp_daily.resample(time = "1M").mean(dim = "time")
anom_months = oisst_anoms_daily.resample(time = "1M").mean(dim = "time")

In [ ]:
oisst_months.sst.isel(time = 5).plot()

In [32]:
anom_months.sst.isel(time = 5).plot()

In [33]:
# Combine to same Dataset
month_tidy = xr.merge([oisst_months, anom_months.rename({"sst": "sst_anom"})])


# Set Attributes: TBD
month_tidy.attrs = {
    "Conventions"   : "CF-1.5",
    "title"         : "NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surface Temperature (OISST) Analysis, Version 2.1",
    "institution"   : "NOAA/National Centers for Environmental Information",
    "source"        : "NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/",
    "References"    : "https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html",
    "dataset_title" : "NOAA Daily Optimum Interpolation Sea Surface Temperature, Monthly Averages",
    "version"       : "Version 2.1",
    "comment"       : "Reynolds, et al.(2007) Daily High-Resolution-Blended Analyses for Sea Surface Temperature (available at https://doi.org/10.1175/2007JCLI1824.1). Banzon, et al.(2016) A long-term record of blended satellite and in situ sea-surface temperature for climate monitoring, modeling and environmental studies (available at https://doi.org/10.5194/essd-8-165-2016). Huang et al. (2020) Improvements of the Daily Optimum Interpolation Sea Surface Temperature (DOISST) Version v02r01, submitted.Climatology is based on 1971-2000 OI.v2 SST. Satellite data: Pathfinder AVHRR SST and Navy AVHRR SST. Ice data: NCEP Ice and GSFC Ice. Data less than 15 days old may be subject to revision."
    
  }
    
# Time attrs
month_tidy.time.attrs =  {
    'long_name' : 'Month',
    'delta_t'   : '0000-01-00 00:00:00',
    'avg_period': '0000-01-00 00:00:00',
    'axis'      : 'T'
}


# Last check:
month_tidy

<xarray.Dataset>
Dimensions:   (time: 487, lat: 720, lon: 1440)
Coordinates:
  * time      (time) datetime64[ns] 1981-09-30 1981-10-31 ... 2022-03-31
  * lat       (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon       (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    sst       (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sst_anom  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>

In [34]:
# Save out monthly netcdf
out_path = f'{box_root}RES_Data/OISST/oisst_mainstays/monthly_averages/oisst_monthly.nc'
month_tidy.to_netcdf(out_path)
print("Saving Monthly Averages")

Saving Monthly Averages
